# Movielens data manipulation

Load movilens datasets to provide movie Recommendation based on movie id.


# Import dependencies


In [1]:
import pandas as pd
import numpy as np
import datetime
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

# Load data


In [2]:
genome_scores_data = pd.read_csv('./data/movielens/genome-scores.csv')
genome_scores_data.head()

,movieId,tagId,relevance
0,1,1,0.02900
1,1,2,0.02375
2,1,3,0.05425
3,1,4,0.06875
4,1,5,0.16000


In [3]:
movies_data = pd.read_csv('./data/movielens/movies.csv')
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_data = pd.read_csv('./data/movielens/ratings.csv')
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


# Prepare data for movie matrix

Create 3 dataframes and calculate 3 cosine similarities for each of them. Then they are mixed to obtain the movie matrix with this formula: `mov_tag_df*0.5+mov_genres_df*0.25+mov_rating_df*0.25`.


# mov_tag_df

Data is pivoted to compare moviea through the tags.


In [5]:
scores_pivot = genome_scores_data.pivot_table(index = ["movieId"],columns = ["tagId"],values = "relevance").reset_index() # compare movie throgh the tags
del genome_scores_data
scores_pivot.head()

tagId,movieId,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,1,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,0.25200,0.22750,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
1,2,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,0.10100,0.08225,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
2,3,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,0.14150,0.04075,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
3,4,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,0.07475,0.03750,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
4,5,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,0.07375,0.02825,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575


# Join "scores_pivot" and "movies_data"

These two dataframes are joined to get all movieIds.


In [6]:
mov_tag_df = movies_data.merge(scores_pivot, left_on='movieId', right_on='movieId', how='left') # get all movieIds
del scores_pivot

mov_tag_df = mov_tag_df.fillna(0) # fill null values
mov_tag_df = mov_tag_df.drop(['title','genres'], axis = 1) # drop clumns not used

mov_tag_df.head()

,movieId,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,1,0.02900,0.02375,0.05425,0.06875,0.16000,0.19525,0.07600,0.25200,0.22750,...,0.03775,0.02250,0.04075,0.03175,0.12950,0.04550,0.02000,0.03850,0.09125,0.02225
1,2,0.03625,0.03625,0.08275,0.08175,0.10200,0.06900,0.05775,0.10100,0.08225,...,0.04775,0.02050,0.01650,0.02450,0.13050,0.02700,0.01825,0.01225,0.09925,0.01850
2,3,0.04150,0.04950,0.03000,0.09525,0.04525,0.05925,0.04000,0.14150,0.04075,...,0.05800,0.02375,0.03550,0.02125,0.12775,0.03250,0.01625,0.02125,0.09525,0.01750
3,4,0.03350,0.03675,0.04275,0.02625,0.05250,0.03025,0.02425,0.07475,0.03750,...,0.04900,0.03275,0.02125,0.03675,0.15925,0.05225,0.01500,0.01600,0.09175,0.01500
4,5,0.04050,0.05175,0.03600,0.04625,0.05500,0.08000,0.02150,0.07375,0.02825,...,0.05375,0.02625,0.02050,0.02125,0.17725,0.02050,0.01500,0.01550,0.08875,0.01575


# mov_genres_df

Split genres field for each movies and create columns for each genres.

Define a function to split genres column and check it if it exists or not.


In [7]:
def set_genres(genres,col):
    '''Split genres column and check if it exists or not.'''
    if genres in col.split('|'): return 1
    else: return 0

In [8]:
mov_genres_df = pd.read_csv('./data/movielens/movies.csv')
# split genres field for each movies and create columns for each genres
mov_genres_df["Action"] = mov_genres_df.apply(lambda x: set_genres("Action",x['genres']), axis=1)
mov_genres_df["Adventure"] = mov_genres_df.apply(lambda x: set_genres("Adventure",x['genres']), axis=1)
mov_genres_df["Animation"] = mov_genres_df.apply(lambda x: set_genres("Animation",x['genres']), axis=1)
mov_genres_df["Children"] = mov_genres_df.apply(lambda x: set_genres("Children",x['genres']), axis=1)
mov_genres_df["Comedy"] = mov_genres_df.apply(lambda x: set_genres("Comedy",x['genres']), axis=1)
mov_genres_df["Crime"] = mov_genres_df.apply(lambda x: set_genres("Crime",x['genres']), axis=1)
mov_genres_df["Documentary"] = mov_genres_df.apply(lambda x: set_genres("Documentary",x['genres']), axis=1)
mov_genres_df["Drama"] = mov_genres_df.apply(lambda x: set_genres("Drama",x['genres']), axis=1)
mov_genres_df["Fantasy"] = mov_genres_df.apply(lambda x: set_genres("Fantasy",x['genres']), axis=1)
mov_genres_df["Film-Noir"] = mov_genres_df.apply(lambda x: set_genres("Film-Noir",x['genres']), axis=1)
mov_genres_df["Horror"] = mov_genres_df.apply(lambda x: set_genres("Horror",x['genres']), axis=1)
mov_genres_df["Musical"] = mov_genres_df.apply(lambda x: set_genres("Musical",x['genres']), axis=1)
mov_genres_df["Mystery"] = mov_genres_df.apply(lambda x: set_genres("Mystery",x['genres']), axis=1)
mov_genres_df["Romance"] = mov_genres_df.apply(lambda x: set_genres("Romance",x['genres']), axis=1)
mov_genres_df["Sci-Fi"] = mov_genres_df.apply(lambda x: set_genres("Sci-Fi",x['genres']), axis=1)
mov_genres_df["Thriller"] = mov_genres_df.apply(lambda x: set_genres("Thriller",x['genres']), axis=1)
mov_genres_df["War"] = mov_genres_df.apply(lambda x: set_genres("War",x['genres']), axis=1)
mov_genres_df["Western"] = mov_genres_df.apply(lambda x: set_genres("Western",x['genres']), axis=1)
mov_genres_df["(no genres listed)"] = mov_genres_df.apply(lambda x: set_genres("(no genres listed)",x['genres']), axis=1)

# drop columns which are no needed anymore
mov_genres_df.drop(['title','genres'], axis = 1, inplace=True)
mov_genres_df.head()

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,(no genres listed)
0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# mov_rating_df

Load movies, extract the year value from the title column and create a new one with this value.


In [9]:
import unicodedata

def set_year(title):
    '''Extract year information from "title" field.'''
    year = title.strip()[-5:-1]
    if year.isdigit(): return int(year)
    else: return 1800


movies = pd.read_csv('./data/movielens/movies.csv')
movies = movies_data.drop('genres', axis = 1)

movies['year'] = movies.apply(lambda x: set_year(x['title']), axis=1)
movies.head()

,movieId,title,year
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


# Tag the year

Movies are labeled with a number that define the year range of the production.


In [10]:
def set_year_group(year):
    '''Groups the years through 0-6.'''
    if (year < 1900): return 0
    elif (1900 <= year <= 1975): return 1
    elif (1976 <= year <= 1995): return 2
    elif (1996 <= year <= 2003): return 3
    elif (2004 <= year <= 2009): return 4
    elif (2005 <= year <= 2015): return 5
    elif (2016 <= year): return 6
    else: return 0
    
movies['year_group'] = movies.apply(lambda x: set_year_group(x['year']), axis=1)

movies.drop(['title','year'], axis = 1, inplace=True)

# group years and rating counts to reduce the scale and increase the similarity calculation
agg_movies_rat = ratings_data.groupby(['movieId']).agg({'rating': [np.size, np.mean]}).reset_index()
# del ratings_data

agg_movies_rat.columns = ['movieId','rating_counts', 'rating_mean']
agg_movies_rat.head()

,movieId,rating_counts,rating_mean
0,1,68469.0,3.886649
1,2,27143.0,3.246583
2,3,15585.0,3.173981
3,4,2989.0,2.874540
4,5,15474.0,3.077291


# Calculate the mean

Calculate the mean and counts the ratings for each movies and then merge it with movies df.


In [11]:
def set_rating_group(rating_counts):
    '''Group rating counts.'''
    if (rating_counts <= 1): return 0
    elif (2 <= rating_counts <= 10): return 1
    elif (11 <= rating_counts <= 100): return 2
    elif (101 <= rating_counts <= 1000): return 3
    elif (1001 <= rating_counts <= 5000): return 4
    elif (5001 <= rating_counts): return 5
    else: return 0
    
agg_movies_rat['rating_group'] = agg_movies_rat.apply(lambda x: set_rating_group(x['rating_counts']), axis=1)

agg_movies_rat.drop('rating_counts', axis = 1, inplace=True)

mov_rating_df = movies.merge(agg_movies_rat, left_on='movieId', right_on='movieId', how='left')
del movies, agg_movies_rat

mov_rating_df = mov_rating_df.fillna(0)

mov_rating_df.head()

,movieId,year_group,rating_mean,rating_group
0,1,2,3.886649,5.0
1,2,2,3.246583,5.0
2,3,2,3.173981,5.0
3,4,2,2.874540,4.0
4,5,2,3.077291,5.0


# Data reduction

To reduce memory usage, only movies produced after 2016 are considered. This df is used to remove not considered movies from previous dfs.


In [12]:
# print(mov_rating_df.head())
rm_index = mov_rating_df[mov_rating_df['year_group'] > 5]
rm_index = rm_index.drop(['year_group','rating_mean','rating_group'], axis = 1)
rm_index.head()

,movieId
27548,122888
27549,122890
27551,122894
27552,122896
27553,122898


In [13]:
movies_data = movies_data.merge(rm_index, left_on='movieId', right_on='movieId', how='right')
# movies_data.to_csv('./data/output/movies_data.csv', index=False)

In [14]:
mov_tag_df = mov_tag_df.merge(rm_index, left_on='movieId', right_on='movieId', how='right')
print(mov_tag_df.head())
mov_tag_df = mov_tag_df.set_index('movieId')
mov_tag_df.head()

   movieId        1        2        3        4        5        6        7  \
0   122888  0.01950  0.01650  0.02700  0.06500  0.08825  0.16450  0.03100   
1   122890  0.02975  0.02525  0.04775  0.07525  0.03925  0.02750  0.01075   
2   122894  0.00000  0.00000  0.00000  0.00000  0.00000  0.00000  0.00000   
3   122896  0.02650  0.02525  0.16525  0.03000  0.02550  0.01800  0.01075   
4   122898  0.06450  0.05800  0.05275  0.06075  0.08975  0.03025  0.03150   

         8        9  ...     1119     1120     1121     1122     1123  \
0  0.03375  0.01150  ...  0.04375  0.02250  0.01575  0.01150  0.15225   
1  0.06475  0.02875  ...  0.06200  0.06850  0.01275  0.00775  0.11300   
2  0.00000  0.00000  ...  0.00000  0.00000  0.00000  0.00000  0.00000   
3  0.05075  0.01450  ...  0.04350  0.02325  0.02300  0.01200  0.20550   
4  0.14200  0.05675  ...  0.06025  0.08525  0.03425  0.02600  0.21100   

      1124     1125     1126     1127     1128  
0  0.05250  0.01500  0.01625  0.12300  0.02200  


,1,2,3,4,5,6,7,8,9,10,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
movieId,,,,,,,,,,,,,,,,,,,,,
122888,0.01950,0.01650,0.02700,0.06500,0.08825,0.16450,0.03100,0.03375,0.01150,0.20925,...,0.04375,0.02250,0.01575,0.01150,0.15225,0.05250,0.01500,0.01625,0.12300,0.02200
122890,0.02975,0.02525,0.04775,0.07525,0.03925,0.02750,0.01075,0.06475,0.02875,0.06350,...,0.06200,0.06850,0.01275,0.00775,0.11300,0.10300,0.04000,0.01250,0.17275,0.02975
122894,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
122896,0.02650,0.02525,0.16525,0.03000,0.02550,0.01800,0.01075,0.05075,0.01450,0.05025,...,0.04350,0.02325,0.02300,0.01200,0.20550,0.06250,0.01475,0.01425,0.24000,0.02475
122898,0.06450,0.05800,0.05275,0.06075,0.08975,0.03025,0.03150,0.14200,0.05675,0.02825,...,0.06025,0.08525,0.03425,0.02600,0.21100,0.07925,0.03400,0.03025,0.15850,0.03050


In [15]:
mov_genres_df = mov_genres_df.merge(rm_index, left_on='movieId', right_on='movieId', how='right')
print(mov_genres_df.head())
mov_genres_df = mov_genres_df.set_index('movieId')
mov_genres_df.head()

   movieId  Action  Adventure  Animation  Children  Comedy  Crime  \
0   122888       0          0          0         0       0      0   
1   122890       1          1          0         0       0      0   
2   122894       1          1          0         0       0      0   
3   122896       0          0          0         0       0      0   
4   122898       1          1          0         0       0      0   

   Documentary  Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  \
0            0      0        0          0       0        0        0        0   
1            0      0        1          0       0        0        0        0   
2            0      0        1          0       0        0        0        0   
3            0      0        0          0       0        0        0        0   
4            0      0        0          0       0        0        0        0   

   Sci-Fi  Thriller  War  Western  (no genres listed)  
0       0         0    0        0               

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,
122888,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
122890,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
122894,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
122896,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
122898,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [16]:
mov_rating_df = mov_rating_df.merge(rm_index, left_on='movieId', right_on='movieId', how='right')
print(mov_rating_df.head())
mov_rating_df = mov_rating_df.set_index('movieId')
# mov_rating_df.drop(rm_index_p, inplace=True)
mov_rating_df.head()

   movieId  year_group  rating_mean  rating_group
0   122888           6     2.849162           3.0
1   122890           6     3.250538           4.0
2   122894           6     2.451613           2.0
3   122896           6     3.253650           4.0
4   122898           6     2.952278           3.0


,year_group,rating_mean,rating_group
movieId,,,
122888,6,2.849162,3.0
122890,6,3.250538,4.0
122894,6,2.451613,2.0
122896,6,3.253650,4.0
122898,6,2.952278,3.0


# movies matrix

Create 3 different datasets to calculate the cosine similarity.


In [17]:
# cosine similarity for mov_tag_df
cos_tag = cosine_similarity(mov_tag_df.values)*0.5

# cosine similarity for mov_genres_df
cos_genres = cosine_similarity(mov_genres_df.values)*0.25
del mov_genres_df

# cosine similarity for mov_rating_df
cos_rating = cosine_similarity(mov_rating_df.values)*0.25
del mov_rating_df

# mix
cos = cos_tag+cos_genres+cos_rating

cols = mov_tag_df.index.values
inx = mov_tag_df.index
movies_sim = pd.DataFrame(cos, columns=cols, index=inx)
# movies_sim.to_csv('./data/output/movies_sim.csv', index=False)
movies_sim.head()

del mov_tag_df, cols, inx

# Create movie_similarity df

Define a function to return the 5 most similar movies based on a moviId.


In [18]:
def get_similar(movieId):
    df = movies_sim.loc[movies_sim.index == movieId].reset_index(). \
            melt(id_vars='movieId', var_name='sim_moveId', value_name='relevance'). \
            sort_values('relevance', axis=0, ascending=False)[1:6]
    return df
    
movies_similarity = pd.DataFrame(columns=['movieId','sim_movieId','relevance'])
print(movies_similarity)

Empty DataFrame
Columns: [movieId, sim_movieId, relevance]
Index: []


#  Find the 5 most similar movies


In [19]:
for x in movies_sim.index.tolist():
    movies_similarity = movies_similarity.append(get_similar(x))
    
movies_similarity = movies_similarity.drop(['sim_movieId'], axis = 1)
print(movies_similarity)

movies_similarity.to_csv('./data/output/movies_similarity.csv', index=False)

     movieId  relevance sim_moveId
3320  122888   0.833331     183909
91    122888   0.830191     152037
29    122888   0.827432     138212
44    122888   0.819126     143387
3     122888   0.817273     122896
...      ...        ...        ...
2931  193886   0.500000     181205
1849  193886   0.500000     172539
588   193886   0.499976     162666
2580  193886   0.499968     178719
582   193886   0.499926     162650

[24855 rows x 3 columns]


# Try movie Recommendation

In [20]:
def movie_recommender(movieId):
    df = movies_sim.loc[movies_sim.index == movieId].reset_index(). \
            melt(id_vars='movieId', var_name='sim_moveId', value_name='relevance'). \
            sort_values('relevance', axis=0, ascending=False)[1:6]
    df['sim_moveId'] = df['sim_moveId'].astype(int)
    sim_df = movies_data.merge(df, left_on='movieId', right_on='sim_moveId', how='inner'). \
                sort_values('relevance', axis=0, ascending=False). \
                loc[: , ['movieId_y','title','genres']]. \
                rename(columns={ 'movieId_y': "movieId" })
    return sim_df

# Recommendation for Deadpool, Action|Adventure|Comedy|Sci-Fi


In [21]:
movie_recommender(122904)

,movieId,title,genres
2,122904,Thor: Ragnarok (2017),Action|Adventure|Sci-Fi
4,122904,Deadpool 2 (2018),Action|Comedy|Sci-Fi
1,122904,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi
3,122904,Guardians of the Galaxy 2 (2017),Action|Adventure|Sci-Fi
0,122904,Black Panther (2017),Action|Adventure|Sci-Fi


# Recommendation for GODZILLA: City on the Edge of Battle, Action|Animation|Sci-Fi

In [22]:
movie_recommender(191235)

,movieId,title,genres
0,191235,PSYCHO-PASS: The Movie (2016),Action|Animation|Sci-Fi
3,191235,Blade Runner: Black Out 2022 (2017),Action|Animation|Sci-Fi
1,191235,Teen Titans: The Judas Contract (2017),Action|Animation|Sci-Fi
2,191235,Starship Troopers: Traitor of Mars (2017),Action|Animation|Sci-Fi
4,191235,Mazinger Z: Infinity (2017),Action|Animation|Sci-Fi


# Prepare movies dataset with year and tmdbId


In [23]:
# prepare movies dataset
movies_df = movies_data.drop('genres', axis = 1)

#calculate mean of ratings for each movies
agg_rating_avg = ratings_data.groupby(['movieId']).agg({'rating': np.mean}).reset_index()

agg_rating_avg.columns = ['movieId', 'rating_mean']

movies_df = movies_df.merge(agg_rating_avg, left_on='movieId', right_on='movieId', how='left')
print(movies_df)

      movieId                                              title  rating_mean
0      122888                                     Ben-hur (2016)     2.849162
1      122890                                    Warcraft (2016)     3.250538
2      122894                                    Avatar 2 (2016)     2.451613
3      122896  Pirates of the Caribbean: Dead Men Tell No Tal...     3.253650
4      122898                              Justice League (2017)     2.952278
...       ...                                                ...          ...
4966   193866                       Tales from the Hood 2 (2018)     1.000000
4967   193872                             Room Laundering (2018)     4.000000
4968   193880                           Her Name Was Mumu (2016)     2.000000
4969   193882                                       Flora (2017)     2.000000
4970   193886                                        Leal (2018)     3.250000

[4971 rows x 3 columns]


## Extract delete year from title and save it in a new column

In [24]:
def del_year(title):
    '''Delete year information from "title" field.'''
    new_title = title.strip()[:-7]
    return new_title
    
movies_df['year'] = movies_df.apply(lambda x: set_year(x['title']), axis=1)
movies_df['title'] = movies_df.apply(lambda x: del_year(x['title']), axis=1)
print(movies_df)

      movieId                                             title  rating_mean  \
0      122888                                           Ben-hur     2.849162   
1      122890                                          Warcraft     3.250538   
2      122894                                          Avatar 2     2.451613   
3      122896  Pirates of the Caribbean: Dead Men Tell No Tales     3.253650   
4      122898                                    Justice League     2.952278   
...       ...                                               ...          ...   
4966   193866                             Tales from the Hood 2     1.000000   
4967   193872                                   Room Laundering     4.000000   
4968   193880                                 Her Name Was Mumu     2.000000   
4969   193882                                             Flora     2.000000   
4970   193886                                              Leal     3.250000   

      year  
0     2016  
1     2016  


## Add tmdbId column

In [25]:
links = pd.read_csv('./data/movielens/links.csv')
print(links)

       movieId   imdbId    tmdbId
0            1   114709     862.0
1            2   113497    8844.0
2            3   113228   15602.0
3            4   114885   31357.0
4            5   113041   11862.0
...        ...      ...       ...
58093   193876    38566   78251.0
58094   193878  1754787   87558.0
58095   193880  5847740  422666.0
58096   193882  4453756  454439.0
58097   193886  7606620  540871.0

[58098 rows x 3 columns]


## Merge movies_df and links, delete 'imdbId' column and convert 'tmvdId' from float to int

In [26]:
movies_df = movies_df.merge(links, left_on='movieId', right_on='movieId', how='left')
del(links)

movies_df = movies_df.drop(['imdbId'], axis = 1)
movies_df['tmdbId'] = movies_df['tmdbId'].astype(int) # convert value to int from float

print(movies_df)

      movieId                                             title  rating_mean  \
0      122888                                           Ben-hur     2.849162   
1      122890                                          Warcraft     3.250538   
2      122894                                          Avatar 2     2.451613   
3      122896  Pirates of the Caribbean: Dead Men Tell No Tales     3.253650   
4      122898                                    Justice League     2.952278   
...       ...                                               ...          ...   
4966   193866                             Tales from the Hood 2     1.000000   
4967   193872                                   Room Laundering     4.000000   
4968   193880                                 Her Name Was Mumu     2.000000   
4969   193882                                             Flora     2.000000   
4970   193886                                              Leal     3.250000   

      year  tmdbId  
0     2016  271969

# Create genres dataset

In [27]:
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
    "(no genres listed)"
]

genres_df = pd.DataFrame(genres, columns=['genre'])
# genres_df.to_csv('./data/output/genres.csv', index=False)
genres_df.head()

,genre
0,Action
1,Adventure
2,Animation
3,Children
4,Comedy


# Create movies_genres dataset with multiple rows for each movies based on genres

In [28]:
movies_genres_df = movies_data.drop('title', axis = 1)

def get_movie_genres(movieId):
    '''Split genres field.'''
    movie = movies_genres_df[movies_genres_df['movieId']==movieId]
    genres = movie['genres'].tolist()
    df = pd.DataFrame([b for a in [i.split('|') for i in genres] for b in a], columns=['genres'])
    df.insert(loc=0, column='movieId', value=movieId)
    return df

# extract genres for each movie
movies_genres=pd.DataFrame(columns=['movieId','genres'])

for x in movies_genres_df['movieId'].tolist():
    movies_genres=movies_genres.append(get_movie_genres(x))

# movies_genres_df.to_csv('./data/output/movies_genres.csv', index=False)
print(movies_genres)

   movieId              genres
0   122888  (no genres listed)
0   122890              Action
1   122890           Adventure
2   122890             Fantasy
0   122894              Action
..     ...                 ...
2   193882              Horror
3   193882              Sci-Fi
0   193886              Action
1   193886               Crime
2   193886               Drama

[8270 rows x 2 columns]


# Get recommendation movies with their information

In [29]:
movies_genres_df = movies_df.merge(movies_genres_df, left_on='movieId', right_on='movieId', how='left')
print(movies_genres_df)

movies_genres_df.to_csv('./data/output/movies.csv', index=False)

      movieId                                             title  rating_mean  \
0      122888                                           Ben-hur     2.849162   
1      122890                                          Warcraft     3.250538   
2      122894                                          Avatar 2     2.451613   
3      122896  Pirates of the Caribbean: Dead Men Tell No Tales     3.253650   
4      122898                                    Justice League     2.952278   
...       ...                                               ...          ...   
4966   193866                             Tales from the Hood 2     1.000000   
4967   193872                                   Room Laundering     4.000000   
4968   193880                                 Her Name Was Mumu     2.000000   
4969   193882                                             Flora     2.000000   
4970   193886                                              Leal     3.250000   

      year  tmdbId                     

In [30]:
id = 122904

def get_movie_info(id):
    '''Get information of a movie with 'movieId'.'''
    query = 'movieId == {}'.format(id)
    movie = movies_genres_df.query(query)
    return movie


get_movie_info(id)

,movieId,title,rating_mean,year,tmdbId,genres
5,122904,Deadpool,3.859245,2016,293660,Action|Adventure|Comedy|Sci-Fi


In [31]:
def get_recommendation_movies(id):
    '''Get the movie recommended for a specific movie with 'movieId'.'''
    query = 'movieId == {}'.format(id)
    recommended_movies = movies_similarity.query(query)
    recommended_movies = recommended_movies.drop(['movieId'], axis = 1)
    for index, row in recommended_movies.iterrows():
        df = get_movie_info(row['sim_moveId'])
        df = df.assign(relevance = row['relevance'])
        print(df)

get_recommendation_movies(id)


   movieId           title  rating_mean  year  tmdbId  \
9   122916  Thor: Ragnarok     3.850147  2017  284053   

                    genres  relevance  
9  Action|Adventure|Sci-Fi   0.910491  
      movieId       title  rating_mean  year  tmdbId                genres  \
3993   187593  Deadpool 2     3.838947  2018  383498  Action|Comedy|Sci-Fi   

      relevance  
3993   0.902662  
   movieId                            title  rating_mean  year  tmdbId  \
8   122912  Avengers: Infinity War - Part I     3.956709  2018  299536   

                    genres  relevance  
8  Action|Adventure|Sci-Fi   0.897366  
    movieId                      title  rating_mean  year  tmdbId  \
10   122918  Guardians of the Galaxy 2     3.859875  2017  283995   

                     genres  relevance  
10  Action|Adventure|Sci-Fi   0.893545  
   movieId          title  rating_mean  year  tmdbId                   genres  \
6   122906  Black Panther     3.684026  2017  284054  Action|Adventure|Sci-Fi   
